## Ввод таблицы из json


In [1]:
import json
import pandas as pd
from PIL import Image
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from scipy.optimize import linprog

import numpy as np

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloader = drive.CreateFile({'id':'1JtJGkWF8ZuhbWVsi6mXjtHlwXDRni-Yj'})
downloader.GetContentFile('test.json')
downloader = drive.CreateFile({'id':'1G8CN4XYwiO7FCgAPTMB_ZnKo3MbFpfkh'})
downloader.GetContentFile('test2.json')
downloader = drive.CreateFile({'id':'1T2qkFzlyQZDbBPau0E5iio7hIr5XoamR'})
downloader.GetContentFile('test3.json')
downloader = drive.CreateFile({'id':'1fGjBW12r66i8mJrvdMqKxK_pD1DD3h-l'})
downloader.GetContentFile('test4.json')
downloader = drive.CreateFile({'id':'1SQ1NdixnIzEwNacwUNpx-SgfweT1A9zQ'})
downloader.GetContentFile('test5.json')
downloader = drive.CreateFile({'id':'15Msrsp5SRJieMNUk-aTlCrhYwM8eYnXj'})
downloader.GetContentFile('lab1.py')

In [2]:
def parse_json(json_name):
  with open(json_name) as json_file:
    data = json.load(json_file)
    strategies_A = data['strategies_A']
    strategies_B = data['strategies_B']
    payments = data['payments']
    return strategies_A, strategies_B, payments

## Упрощение путём анализа доминирующих статегий

Упрощаем строки

In [3]:
def dominance_row(payment_matrix):
  payment_matrix.drop_duplicates()
  dominating_filter = (payment_matrix.values[:, None] <= payment_matrix.values).all(axis=2).sum(axis=1) == 1
  payment_matrix = payment_matrix[dominating_filter]
  return payment_matrix

Упрощаем столбцы

In [4]:
def dominance_colomn(payment_matrix):
  payment_matrix_T = payment_matrix.T
  dominating_filter = (payment_matrix_T.values[:, None] >= payment_matrix_T.values).all(axis=2).sum(axis=1) == 1
  payment_matrix_T = payment_matrix_T[dominating_filter]
  return payment_matrix_T.T

## Решение игры в чистых стратегиях

Для нахождения решения игры в чистых стратегиях можно использовать метод минимакса. Для каждой строки матрицы платежей нужно найти минимальное значение и выбрать максимальное из них. Затем сделайть то же самое для каждого столбца матрицы платежей, но на этот раз найти максимальное значение и выбирать минимальное из них. Когда найденные значения совпадают, это является оптимальными стратегиями игры.

In [5]:
def pure_strategy(payment_matrix):
  row_min = payment_matrix.min(axis=1)
  saddle_point_row = row_min.max()

  column_max = payment_matrix.max(axis=0)
  saddle_point_column = column_max.min()

  if saddle_point_row == saddle_point_column:
    optimal_row_strategy = row_min.idxmax()
    optimal_column_strategy = column_max.idxmin()
    game_value = saddle_point_row
    print("Оптимальная стратегия игры:")
    print("Стратегия игрока A:", optimal_row_strategy)
    print("Стратегия игрока B:", optimal_column_strategy)
    print("Цена игры:", game_value)
    return True
  else:
    print("Решение игры в чистых стратегиях отсутствует.")
    return False

## Смешанные стратегии

In [6]:
def mixed_strategy(payment_matrix):

  # Для первого игрока

  # Создаем матрицу коэффициентов для симплекс-метода
  matrix_ = payment_matrix.values.T
  num_columns = len(payment_matrix[0])
  c = [1] * num_columns
  A = []
  b = []
  for column in matrix_:
    A.append([-v for v in column])
    b.append(-1)

  # Решаем задачу симплекс-методом (поиск минимума функции F(x) при ограничениях)
  result1 = linprog(c, A_ub=A, b_ub=b, bounds=[(0, None)] * num_columns, method='simplex', options={"disp": True})

  # Для второго игрока

  # Создаем матрицу коэффициентов для симплекс-метода
  matrix_ = payment_matrix.values
  num_columns = len(matrix_[1])
  c = [-1] * num_columns
  A = []
  b = []
  for row in matrix_:
    A.append(row)
    b.append(1)

  # Решаем задачу симплекс-методом (поиск максимума функции Z(y) при ограничениях)
  result2 = linprog(c, A_ub=A, b_ub=b, bounds=[(0, None)] * num_columns, method='simplex', options={"disp": True})

  if result1.success and result2.success:
    # Вычисляем цену игры
    game_value = 1 / abs(result2.fun)

    # Извлекаем результаты
    mixed_strategy_A = game_value * result1.x
    mixed_strategy_B = game_value * result2.x

    print("Смешанные стратегии игры:")
    print("Смешанная стратегия игрока A:", mixed_strategy_A)
    print("Смешанная стратегия игрока B:", mixed_strategy_B)
    print("Цена игры:", game_value)
  else:
    print("Решение в смешанных стратегиях не найдено.")



## Тесты

In [7]:
def solve_game(json_name):
  strategies_A, strategies_B, payments = parse_json(json_name)
  payment_matrix = pd.DataFrame(payments, index=strategies_A, columns=strategies_B)

  payment_matrix = dominance_row(payment_matrix)
  payment_matrix = dominance_colomn(payment_matrix)
  print(payment_matrix)

  if(pure_strategy(payment_matrix)):
    return

  mixed_strategy(payment_matrix)

In [8]:
solve_game("test.json")

     0
2  700
Оптимальная стратегия игры:
Стратегия игрока A: 2
Стратегия игрока B: 0
Цена игры: 700


In [9]:
solve_game("test3.json")

   0  2
1  3  5
2  2 -2
Оптимальная стратегия игры:
Стратегия игрока A: 1
Стратегия игрока B: 0
Цена игры: 3


In [10]:
solve_game("test4.json")

   3
0 -4
1 -1
Оптимальная стратегия игры:
Стратегия игрока A: 1
Стратегия игрока B: 3
Цена игры: -1


In [11]:
solve_game("test5.json")

   0  1
0  2 -1
1  1  3
Решение игры в чистых стратегиях отсутствует.
Optimization terminated successfully.
         Current function value: 0.714286    
         Iterations: 2
Optimization terminated successfully.
         Current function value: -0.714286   
         Iterations: 2
Смешанные стратегии игры:
Смешанная стратегия игрока A: [0.4 0.6]
Смешанная стратегия игрока B: [0.8 0.2]
Цена игры: 1.4000000000000001


<ipython-input-6-dd7ef0db84a0>:16: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result1 = linprog(c, A_ub=A, b_ub=b, bounds=[(0, None)] * num_columns, method='simplex', options={"disp": True})
<ipython-input-6-dd7ef0db84a0>:31: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result2 = linprog(c, A_ub=A, b_ub=b, bounds=[(0, None)] * num_columns, method='simplex', options={"disp": True})


In [12]:
solve_game("test2.json")

   0  1  2  3  4
0  1  2  3  4  5
1  3  2  1  0 -1
2  1  3  5  4  2
3  3  1 -1  0  2
Решение игры в чистых стратегиях отсутствует.
Optimization terminated successfully.
         Current function value: 0.500000    
         Iterations: 6
Optimization terminated successfully.
         Current function value: -0.500000   
         Iterations: 4
Смешанные стратегии игры:
Смешанная стратегия игрока A: [2.46716228e-17 0.00000000e+00 5.00000000e-01 5.00000000e-01]
Смешанная стратегия игрока B: [0.64285714 0.         0.21428571 0.         0.14285714]
Цена игры: 2.0


<ipython-input-6-dd7ef0db84a0>:16: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result1 = linprog(c, A_ub=A, b_ub=b, bounds=[(0, None)] * num_columns, method='simplex', options={"disp": True})
<ipython-input-6-dd7ef0db84a0>:31: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result2 = linprog(c, A_ub=A, b_ub=b, bounds=[(0, None)] * num_columns, method='simplex', options={"disp": True})
